In [69]:
from PIL import ImageGrab
import cv2
import numpy as np
import pytesseract
from pyzbar.pyzbar import decode

In [142]:
try:
    # Получаем изображение из буфера обмена
    image = ImageGrab.grabclipboard()

    # Преобразуем изображение в черно-белое
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

    # Находим границы на изображении
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Находим линии на изображении
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    # Находим угол наклона текста на изображении
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
        # if abs(angle) > 60:
        #     angle = 90 -angle
        # print(angle)
        angles.append(angle)
    angle = np.median(angles)

    # Поворачиваем изображение на найденный угол
    (h, w) = image.size
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(np.array(image), M, (w, h))

    # Распознаем текст на повернутом изображении
    text = pytesseract.image_to_string(rotated, lang='rus')
    print(text)
    # # Находим координаты ограничивающего прямоугольника для текста
    # boxes = pytesseract.image_to_boxes(rotated, lang='rus')
    # for b in boxes.splitlines():
    #     b = b.split(' ')
    #     x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
    #     cv2.rectangle(rotated, (x, h), (w, y), (0, 255, 0), 2)

    # # Кадрируем изображение по ограничивающему прямоугольнику
    # x, y, w, h = cv2.boundingRect(boxes)
    # cropped = rotated[y:y+h, x:x+w]

    # # Отображаем результат
    # cv2.imshow('Cropped Image', cropped)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
except:
    pass

try:
    # Получаем изображение из буфера обмена
    image = ImageGrab.grabclipboard()

    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    # Поиск контуров на изображении
    contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Поиск прямоугольников, ограничивающих контуры
    rects = []
    for contour in contours:
        rect = cv2.minAreaRect(contour)
        rects.append(rect)

    # Поиск угла поворота изображения
    angles = []
    for rect in rects:
        angle = rect[2]
        if angle < -45:
            angle += 90
        angles.append(angle)

    # Поворот изображения на найденный угол
    rotated = cv2.rotate(np.array(image), cv2.ROTATE_90_CLOCKWISE)

    # Декодирование штрих-кода
    decoded = decode(rotated)
    # print(decoded)
    code = decoded[0]
    # Вывод результата
    print(int(code.data))
except:
    pass

try:
    # Получаем изображение из буфера обмена
    image = ImageGrab.grabclipboard()

    # Преобразование изображения в оттенки серого
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    # нахождение контуров
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # нахождение минимального ограничивающего прямоугольника
    rect = cv2.minAreaRect(contours[0])

    # поворот изображения
    angle = rect[2]
    rows, cols = np.array(image).shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    img_rotated = cv2.warpAffine(np.array(image), M, (cols, rows))

    # распознавание штрих-кода
    decoded = decode(cv2.cvtColor(img_rotated, cv2.COLOR_BGR2GRAY))
    print(decoded)
    # вывод результата
    print(decoded[0].data.decode('utf-8'))
except:
    pass


[]
